In [ ]:
import os
from transformers import pipeline
from pathlib import Path
from docx import Document
from tqdm import tqdm
from transformers import AutoTokenizer

TRANSLATION_CONFIG = {
    # Configurazioni del modello
    "model_name": "Helsinki-NLP/opus-mt-it-en",
    "framework": "pt",
    "device": "cpu",
    
    # Configurazioni dei percorsi
    "paths": {
        "input_dir": "it",
        "output_dir": "en"
    }
}

tokenizer = AutoTokenizer.from_pretrained(TRANSLATION_CONFIG["model_name"])


In [ ]:
def translate_docx(input_file, output_file):
    # Inizializza il modello di traduzione
    pipe = pipeline(
        "translation",
        model=TRANSLATION_CONFIG["model_name"],
        tokenizer=tokenizer,
        framework=TRANSLATION_CONFIG["framework"],
        device=TRANSLATION_CONFIG["device"]
    )
    
    # Leggi il documento Word
    doc = Document(input_file)
    
    # Crea un nuovo documento per la traduzione
    translated_doc = Document()
    
    # Traduci ogni paragrafo
    for paragraph in tqdm(doc.paragraphs, desc=f"Traduzione {input_file.name}"):
        if paragraph.text.strip():
            translation = pipe(paragraph.text)
            translated_doc.add_paragraph(translation[0]['translation_text'])
        else:
            translated_doc.add_paragraph('')
    
    # Salva il documento tradotto
    translated_doc.save(output_file)
    print(f"File tradotto con successo: {output_file}")

# Definisci i percorsi delle cartelle
input_dir = Path(TRANSLATION_CONFIG["paths"]["input_dir"])
output_dir = Path(TRANSLATION_CONFIG["paths"]["output_dir"])

# Assicurati che la cartella di output esista
output_dir.mkdir(exist_ok=True)

# Itera su tutti i file nella cartella di input
for input_file in input_dir.glob("*.docx"):
    # Crea il nome del file di output
    output_file = output_dir / f"{input_file.stem}_EN{input_file.suffix}"
    
    # Traduci il file
    translate_docx(input_file, output_file)